In [62]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
import numpy as np

In [2]:
pd.set_option('display.max_rows', 1000)

In [3]:
client_categories=pd.read_csv('docs/client_categories.csv',sep=';')
clients=pd.read_csv('docs/clients_last_2_fixed.csv', sep=';')
full_clients=pd.merge(clients,client_categories,how='left',left_on='categorycode',right_on='category')
stores=pd.read_csv('docs/store_last_2.csv', error_bad_lines=False,encoding= 'windows-1251',sep=';')
transactions=pd.read_csv('docs/transactions_last_2.csv',encoding= 'windows-1251',sep=';',nrows=5000000)

In [10]:
mccs=pd.read_excel('docs/MCC_last.xlsx')

0. Пол, возраст, замужний/нет, резидент/нет, банковская категория 

In [ ]:
mccs['']

In [4]:
full_clients.head(10)

,cnum_,categorycode,gender,age,married_,residenttype,category,description
0,0CCCDO,81,M,32,not_married,R,81,Affluent Banking payroll customer
1,0CCCFO,70,F,42,not_married,R,70,Regular retail banking customer
2,0CCCGC,50,F,33,married,R,50,Corporate customer employee/ payroll scheme
3,0CCCGG,70,M,58,not_married,R,70,Regular retail banking customer
4,0CCCGO,60,M,38,married,R,60,RBA employees
5,0CCCGS,50,M,40,not_married,R,50,Corporate customer employee/ payroll scheme
6,0CCCHG,70,M,30,married,R,70,Regular retail banking customer
7,0CCCHS,80,M,70,married,R,80,Affluent Banking regular customer
8,0CCCNN,70,F,32,not_married,R,70,Regular retail banking customer
9,0CCCNX,50,M,61,married,R,50,Corporate customer employee/ payroll scheme


In [8]:
stores.sample(5)

,store_name,merchant_name
400716,Трц Браво,EJI 0J7TX
483195,Вкусвилл,T3LKT4HH MQSP M
464566,Uber,L01J EJ4W UJYZJ A1HW.L 7CKE1J87C DH
68708,Billa,04HH7 01HX0XJX8XT7 CXK3T7 JL
412801,Uber,L01J JL 910BV YGUEI A1


In [6]:
transactions.head()

,purchdate,amount,mcc,mrchcity,mrchname,cnum
0,2019-11-02 00:00:00,9000000.0,5691,MOSCOW,IA7D1H Z1D1T7,MJNLMQ
1,2019-11-02 00:00:00,9000000.0,5691,MOSCOW,IA7D1H Z1D1T7,MJNLMQ
2,2019-10-25 00:00:00,7716900.0,5691,MOSCOW,HXL4K TL4EEXD,ELETCO
3,2019-11-29 00:00:00,5100000.0,6513,SANKT-PETERBU,7X 1E7HXD H1DKW1IKCL,EL0KSF
4,2019-12-03 00:00:00,5023956.6,5944,MOSKVA,73 7HJXK7 (W7X),MPKIJJ


In [12]:
mccs.head()

,mcc,category
0,742,Pet
1,763,Other
2,780,Maintenance and building
3,1520,Maintenance and building
4,1711,Maintenance and building


1. Туризм
-смотрим количество транзакций
-количество городов, в которых транзакции были

In [14]:
transactions['Строка']=1
clients_unique_cities=transactions.groupby('cnum')['mrchcity'].nunique()
clients_number_of_transactions=transactions.groupby('cnum')['Строка'].sum()
tourism=pd.concat([clients_unique_cities, clients_number_of_transactions], axis=1)
tourism['Коэффициент_туризма']=tourism['mrchcity']/tourism['Строка']

In [15]:
full_clients=pd.merge(full_clients,tourism[['Коэффициент_туризма']],how='left',left_on='cnum_',right_index=True)

2. Интересы

In [17]:
full_clients['description'].unique()

array(['Affluent Banking payroll customer',
       'Regular retail banking customer',
       'Corporate customer employee/ payroll scheme', 'RBA employees',
       'Affluent Banking regular customer',
       'Premium Direct Regular Customer',
       'Premium Direct Payroll Customer',
       'Corporate customer manager/ regular',
       'Affluent Banking potential payroll customer',
       'Private Banking customers walk-in',
       'Private banking customers payroll',
       'Regular retail banking customer (family)',
       'Affluent Banking potential regular customer',
       'RBA employees (family)', 'Corporate customer manager/ VIP',
       'High income individuals (family)',
       'Corporate customer employee/ payroll scheme (family)',
       'High income individuals',
       'Corporate customer manager/ regular (family)'], dtype=object)

In [19]:
mccs['category'].unique()

array(['Pet', 'Other', 'Maintenance and building', 'Design, print',
       'Tourism', 'Railway and sea transport', 'Local and autotransport',
       'Medical', 'Taxi', 'Postal services, delivery', 'Active rest',
       'Auto', 'Soft', 'Providers', 'Money', 'Payments', 'Clothes',
       'Cultural life', 'Furniture, home', 'Discount', 'Supermarkets',
       'Parking&Fuel', 'Kids', 'Sport', 'Services', 'Restaurants',
       'Fast Food', 'Digital goods', 'Alcohol and tobacco', 'Gift, hobby',
       'Legal and insurance', 'Beauty', 'Video games', 'Gambling',
       'Education'], dtype=object)

In [24]:
grouped_tr=transactions[['cnum','mcc','amount']].groupby(['cnum','mcc']).sum()

In [31]:
grouped_tr.sort_values(by='amount',ascending=False,inplace=True)

In [32]:
grouped_tr.loc['ELCBGE']

,amount
mcc,
6011,40084500.0
6211,12939000.0
7299,1000000.0
6012,392300.0
5941,43832.0


In [40]:
len(full_clients['cnum_'].unique())

1355925

In [41]:
len(transactions['cnum'].unique())

854925

In [60]:
grouped_tr

,,amount
cnum,mcc,
ELCBGE,6011,40084500.0
ELFQTA,6011,39545000.0
QTQRJR,6011,35176600.0
QTQKKK,6011,32160000.0
MJNLMQ,5691,28905999.0
...,...,...
ELMC64,5691,9566.0
VFCSSC,4511,9566.0
2GYYYO,9311,9566.0


In [ ]:

_interests={}
i=0
for item in transactions['cnum'].unique()[:100000]:
    _interests.update({item:list(grouped_tr.loc[item].index)[:3]})
#     _interests.update({item:grouped_tr.loc[item].index[0]})
    if(i%10000==0):
        print(i)
    i+=1


In [ ]:
'''
i=0
for item in transactions['cnum'].unique()[100000:]:
    _interests.update({item:list(grouped_tr.loc[item].index)[:3]})
#     _interests.update({item:grouped_tr.loc[item].index[0]})
    if(i%10000==0):
        print(i)
    i+=1

'''

In [ ]:
full_clients['интерес_1']=none
full_clients['интерес_2']=none
full_clients['интерес_3']=none

3. Импульсивность/стабильность



In [64]:
grouped_tr2=transactions[['cnum','purchdate','amount']].groupby(['purchdate','amount']).sum()

In [66]:
grouped_tr2.head()

cnum
purchdate           amount          
2019-09-02 00:00:00 11990.0   JKQQJM
2019-09-03 00:00:00 12190.0   ELUXO0
2019-09-04 00:00:00 132000.0  QKTRMT
2019-09-05 00:00:00 23200.0   EEZODQ
                    50000.0   EEIDFE

In [86]:
_impulsivity={}

for item in transactions['cnum'].unique()[:100]:
    _impulsivity.update(item:np.std([x[1] for x in grouped_tr2[grouped_tr2['cnum']==item].index])/np.mean([x[1] for x in grouped_tr2[grouped_tr2['cnum']==item].index]))
#     print(grouped_tr2[grouped_tr2['cnum']==item].index)
#     print([x[1] for x in grouped_tr2[grouped_tr2['cnum']==item].index])
#     print(np.std([x[1] for x in grouped_tr2[grouped_tr2['cnum']==item].index])/np.mean([x[1] for x in grouped_tr2[grouped_tr2['cnum']==item].index]))
#     grouped_tr2[grouped_tr2['cnum']==item].xs('amount', level=1, drop_level=False)
#     df.loc[(slice(None), slice('amount')), :]
#     _impulsivity.update({item:np.std(list(grouped_tr2.loc[item]))})
    
#     _impulsivity.update(np.std(transactions[transactions['cnum']==item].index))
    
#     print(np.mean(list(transactions[transactions['cnum']==item].index)))


1.5435945889276
4.988951966859915
0.0
0.0
1.960956945788344
0.9745675697022352
0.0
0.7928353807088866
2.795846159744538
3.016703343026194


4. Размер зп? дети?

In [ ]:
'''


'''

5. Кластеризация возраст+зп

6. Разносторонний или нет

In [87]:
clients_unique_stores=transactions.groupby('cnum')['mrchname'].nunique()
customer_behavior=pd.concat([clients_unique_stores, clients_number_of_transactions], axis=1)
tourism['Коэффициент_разносторонности']=customer_behavior['mrchname']/tourism['Строка']

7. Кластеризация

In [ ]:
'''
Темперамент?
возраст, сколько он тратит

богатство, возраст
№ 201115-255614

'''

In [88]:
transactions['строка']=1

In [89]:
grouped_tr3=transactions[['cnum','amount','строка']].groupby(['cnum']).sum()

In [91]:
grouped_tr3['Состоятельность']=grouped_tr3['amount']/grouped_tr3['строка']

In [92]:
grouped_tr3.head()

,amount,строка,Состоятельность
cnum,,,
0CCCDO,165506.00,3,55168.666667
0CCCGC,36290.00,3,12096.666667
0CCCGG,88968.25,4,22242.062500
0CCCGO,82600.00,8,10325.000000
0CCCGS,415783.64,13,31983.356923


In [93]:
full_clients=pd.merge(full_clients,grouped_tr3[['Состоятельность']],how='left',left_on='cnum_',right_index=True)

In [94]:
full_clients.head()

,cnum_,categorycode,gender,age,married_,residenttype,category,description,Коэффициент_туризма,Состоятельность
0,0CCCDO,81,M,32,not_married,R,81,Affluent Banking payroll customer,0.333333,55168.666667
1,0CCCFO,70,F,42,not_married,R,70,Regular retail banking customer,NaN,NaN
2,0CCCGC,50,F,33,married,R,50,Corporate customer employee/ payroll scheme,1.000000,12096.666667
3,0CCCGG,70,M,58,not_married,R,70,Regular retail banking customer,0.500000,22242.062500
4,0CCCGO,60,M,38,married,R,60,RBA employees,0.125000,10325.000000


In [95]:
full_clients_for_cl=full_clients.notna()

In [ ]:
corr = full_clients_for_cl.corr()
corr.style.background_gradient(cmap='coolwarm')

In [97]:
grouped_tr4=transactions[['cnum','строка']].groupby(['cnum']).sum()

In [98]:
grouped_tr4.sort_values(by=['строка'],ascending=False,inplace=True)

In [101]:
list(grouped_tr4.index)[:20]

['EERKDC',
 'ELCBGE',
 'EET9Q4',
 '8YFNOH',
 '0HXSDH',
 'EL6MR2',
 'EEVVH8',
 'JRRMIR',
 'MQPKMI',
 'EL95FB',
 'IMMJKR',
 'EL80P2',
 'TRNLNI',
 'LLPTQL',
 'QTQRJR',
 'ELDTP7',
 'ELFE5U',
 'QHDNOY',
 'EET6PH',
 'E46NJV']

In [102]:
grouped_tr4.head()

,строка
cnum,
EERKDC,503
ELCBGE,483
EET9Q4,382
8YFNOH,361
0HXSDH,358


In [103]:
list(grouped_tr4[grouped_tr4['строка'].isin([1,2])].index)[:10]

['E4FT88',
 'E4FTAN',
 'E4FYNM',
 'E4FTAG',
 'KNGFYH',
 '0XDDGD',
 'E4FTA0',
 'KNGNNG',
 'EL5MRC',
 'KNGHNG']